In [199]:
import warnings
from typing import Iterable, Tuple
import torch
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from IPython.display import clear_output
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.distributions.categorical import Categorical

warnings.filterwarnings("ignore")

In [200]:
with open(r"../../additional_materials/anek_djvu.txt", "r", encoding="utf-8") as f:
    text = f.read()
text[118:500]

'|startoftext|>Друзья мои, чтобы соответствовать вам, я готов сделать над собой усилие и стать лучше. Но тогда и вы станьте немного хуже!\n\n<|startoftext|>- Люся, ты все еще хранишь мой подарок?- Да.- Я думал, ты выкинула все, что со мной связано.- Плюшевый мишка не виноват, что ты ебл@н...\n\n<|startoftext|>- А вот скажи честно, ты во сне храпишь?- Понятие не имею, вроде, нет. От со'

In [201]:
def cut_data(text):
    return text.replace("\n\n", "").split("<|startoftext|>")[1:]

In [202]:
cut_text = cut_data(text)

In [203]:
cut_text[1:6]

['Друзья мои, чтобы соответствовать вам, я готов сделать над собой усилие и стать лучше. Но тогда и вы станьте немного хуже!',
 '- Люся, ты все еще хранишь мой подарок?- Да.- Я думал, ты выкинула все, что со мной связано.- Плюшевый мишка не виноват, что ты ебл@н...',
 '- А вот скажи честно, ты во сне храпишь?- Понятие не имею, вроде, нет. От собственного храпа по крайней мере еще ни разу не просыпался.- Ну, так у жены спроси.- А жена и подавно не знает. У нее странная привычка после замужества возникла: как спать ложится - беруши вставляет.',
 'Поссорилась с мужем. Пока он спал, я мысленно развелась с ним, поделила имущество, переехала, поняла, что жить без него не могу, дала последний шанс, вернулась. В итоге, ложусь спать уже счастливой женщиной.',
 'Если тебя посещают мысли о смерти - это еще полбеды. Беда - это когда смерть посещают мысли о тебе...']

### Задача 1

Обучите RNN/LSTM на данных из классной работы, используя другой токенайзер. Опишите его и свой выбор. Покажите разницу в генерации моделей, обученных с разными токенайзерами.

Токенайзер из библиотеки **sentencepiece**: 
- работает с неразмеченными данными
- разбивает его на подсловные юниты, что позволяет лучше справляться с редкими словами и морфологически сложными языками
- поддерживает несколько методов токенизации, включая BPE (Byte Pair Encoding)

BPE:
- На каждом шаге алгоритм находит наиболее часто встречающуюся пару соседних символов или последовательностей символов и объединяет их в новый символ или токен.
- Обновление: Этот процесс повторяется, пока не будет достигнуто заданное количество токенов или не останется частых пар для объединения.

Преимущества:
- Обработка редких слов: BPE позволяет разбивать редкие слова на более частые подсловные юниты, что улучшает обобщающую способность моделей.
- Гибкость: Позволяет контролировать размер словаря, что важно для управления ресурсами и производительностью моделей.
- Языконезависимость: Может применяться к любому языку, так как не зависит от предварительной сегментации на слова.


In [204]:
   import sentencepiece as spm

   with open(r"../../additional_materials/anek_djvu.txt", "r", encoding="utf-8") as f:
       text = f.read()

   # Обучаем токенайзер
   spm.SentencePieceTrainer.train(input='../../additional_materials/anek_djvu.txt', model_prefix='m', vocab_size=454, model_type='bpe')

   # Загружаем обученный токенайзер
   sp = spm.SentencePieceProcessor();
   sp.load('m.model') # загружаем модель в файл

   # Токенизация текста
   tokens = sp.encode_as_pieces(text[118:500])
   print("Tokens:", tokens)

   # Детокенизация
   detokenized_text = sp.decode_pieces(tokens)
   print("Detokenized text:", detokenized_text)

Tokens: ['▁', '|', 'startoftext', '|>', 'Д', 'ру', 'з', 'ь', 'я', '▁мо', 'и', ',', '▁чтобы', '▁со', 'о', 'т', 'вет', 'ство', 'вать', '▁ва', 'м', ',', '▁я', '▁го', 'то', 'в', '▁с', 'де', 'ла', 'ть', '▁на', 'д', '▁со', 'бо', 'й', '▁у', 'си', 'ли', 'е', '▁и', '▁ста', 'ть', '▁', 'лу', 'ч', 'ше', '.', '▁Н', 'о', '▁то', 'гда', '▁и', '▁вы', '▁ста', 'нь', 'те', '▁не', 'м', 'ного', '▁х', 'уж', 'е', '!', '▁<|', 'startoftext', '|>-', '▁', 'Л', 'ю', 'ся', ',', '▁ты', '▁все', '▁еще', '▁х', 'ра', 'ни', 'шь', '▁мо', 'й', '▁по', 'да', 'ро', 'к', '?-', '▁Да', '.-', '▁Я', '▁ду', 'ма', 'л', ',', '▁ты', '▁вы', 'ки', 'ну', 'ла', '▁все', ',', '▁что', '▁со', '▁м', 'ной', '▁с', 'в', 'я', 'за', 'но', '.-', '▁П', 'лю', 'ше', 'вы', 'й', '▁ми', 'шка', '▁не', '▁ви', 'но', 'ва', 'т', ',', '▁что', '▁ты', '▁е', 'б', 'л', '@', 'н', '...', '▁<|', 'startoftext', '|>-', '▁А', '▁во', 'т', '▁с', 'ка', 'жи', '▁че', 'ст', 'но', ',', '▁ты', '▁во', '▁с', 'не', '▁х', 'ра', 'пи', 'шь', '?-', '▁По', 'ня', 'ти', 'е', '▁не', '▁и', 

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: ../../additional_materials/anek_djvu.txt
  input_format: 
  model_prefix: m
  model_type: BPE
  vocab_size: 454
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_pr

In [205]:
class SPTokenizer:
    def __init__(self, model_path: str):
        self.sp = spm.SentencePieceProcessor()
        self.sp.load(model_path)
    
    @property
    def vocab_size(self):
        return self.sp.get_piece_size()
        
    def encode(self, text):
        return self.sp.encode_as_ids(text)
    
    def decode(self, ids):
        return self.sp.decode_ids(ids)
    
    def encode_symbol(self, symbol):
        return self.sp.piece_to_id(symbol)
    
    def decode_symbol(self, id):
        return self.sp.id_to_piece(id)


In [206]:
tokenizer = SPTokenizer('m.model')

# Пример использования
text = "Привет, как дела?"
encoded_text = tokenizer.encode(text)
print("Encoded text:", encoded_text)

decoded_text = tokenizer.decode(encoded_text)
print("Decoded text:", decoded_text)

Encoded text: [95, 52, 306, 383, 154, 143, 44, 404]
Decoded text: Привет, как дела?


In [207]:
class JokesDataset(Dataset):
    def __init__(self, tokenizer, cut_text, max_len: int = 512):
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.cut_text = cut_text
        self.pad_index = self.tokenizer.encode_symbol("<pad>")
        
    def __getitem__(self, item):
        # pad your sequence and make a final sample. You can skip padding and pad sequences with torch special method.
        #text = self.cut_text[item]
        encoded = self.tokenizer.encode(text)  # Токенизируем строку

        if len(encoded) > self.max_len:
            encoded = encoded[:self.max_len]

        # Создаем тензор padded с размером max_len
        padded = torch.full((self.max_len, ), self.pad_index, dtype=torch.long)
        # Заполняем тензор padded значениями из encoded
        padded[:len(encoded)] = torch.tensor(encoded)

        return padded, len(encoded)
    
    def __len__(self):
        return len(self.cut_text)

In [208]:
import torch
from torch import nn

class LSTM(nn.Module):
    def __init__(self, tokenizer, embedding_dim=128, hidden_dim=256, num_layers=2, drop_prob=0.5):
        super(LSTM, self).__init__()
        self.tokenizer = tokenizer
        self.vocab_size = tokenizer.vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.embedding = nn.Embedding(self.vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        # Полносвязный слой
        self.fc = nn.Linear(hidden_dim, self.vocab_size)

    def forward(self, x, hidden):
        # Пропускаем входные данные через слой эмбеддингов
        x = self.embedding(x)
        
        # Пропускаем через LSTM
        lstm_out, hidden = self.lstm(x, hidden)
        
        # Применяем dropout
        out = self.dropout(lstm_out)
        
        # Пропускаем через полносвязный слой
        out = self.fc(out)
        
        # Возвращаем выход и скрытое состояние
        return out, hidden

    def init_hidden(self, batch_size, device):
        # Инициализируем скрытое состояние нулями
        weight = next(self.parameters()).data
        hidden = (weight.new_zeros(self.num_layers, batch_size, self.hidden_dim).to(device),
                  weight.new_zeros(self.num_layers, batch_size, self.hidden_dim).to(device))
        return hidden

    def inference(self, start_text, device, max_length=100):
        self.eval()  # Переключаем модель в режим оценки
        tokens = self.tokenizer.encode(start_text)
        tokens_tensor = torch.tensor(tokens).unsqueeze(0).to(device)
        hidden = self.init_hidden(1, device)

        generated_text = start_text

        for _ in range(max_length):
            output, hidden = self.forward(tokens_tensor, hidden)
            probabilities = torch.softmax(output[:, -1], dim=-1)
            next_token = torch.argmax(probabilities, dim=-1).item()
            tokens_tensor = torch.cat((tokens_tensor, torch.tensor([[next_token]]).to(device)), dim=1)

            if next_token == self.tokenizer.encode_symbol("<eos>"):
                break

            generated_text += self.tokenizer.decode([next_token])

        return generated_text

Зададим параметры для обучения модели

In [209]:
batch_size = 6
seq_length = 128
n_hidden = 64
n_layers = 2
drop_prob = 0.1
lr = 0.1
embedding_dim = 128

In [210]:
def training_step(
    model: LSTM,
    train_batch: Tuple[torch.Tensor, torch.Tensor],
    vocab_size: int,
    criterion: nn.Module,
    optimizer,
    device="cpu"
) -> torch.Tensor:
    
    batch_size, seq_len = train_batch[0].shape
    
    # Переводим данные на устройство
    inputs = train_batch[0].to(device)
    targets = train_batch[0].to(device)  # Изменено: используем inputs для targets

    optimizer.zero_grad()
    
    # Прямой проход
    output = model(inputs, torch.tensor([seq_len] * batch_size).to(device))
    
    # Изменено: Преобразуем выходные данные в нужную форму
    logits = output.view(-1, vocab_size)
    targets = targets.view(-1)
    
    # Вычисляем функцию потерь
    loss = criterion(logits, targets)
    
    # Обратный проход и оптимизация
    loss.backward()
    optimizer.step()
    
    return loss.item()

In [211]:
'''# Пример использования
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = WordLSTM(tokenizer).to(device)

# Инициализация скрытого состояния
batch_size = 32
hidden = model.init_hidden(batch_size, device)

# Пример входных данных
input_data = torch.randint(0, tokenizer.vocab_size, (batch_size, 10)).to(device)

# Прямой проход
output, hidden = model(input_data, hidden)
print(output.shape)  # Ожидаемый размер: (batch_size, sequence_length, vocab_size)
'''

'# Пример использования\ndevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")\nmodel = WordLSTM(tokenizer).to(device)\n\n# Инициализация скрытого состояния\nbatch_size = 32\nhidden = model.init_hidden(batch_size, device)\n\n# Пример входных данных\ninput_data = torch.randint(0, tokenizer.vocab_size, (batch_size, 10)).to(device)\n\n# Прямой проход\noutput, hidden = model(input_data, hidden)\nprint(output.shape)  # Ожидаемый размер: (batch_size, sequence_length, vocab_size)\n'

In [212]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = LSTM(tokenizer, embedding_dim, n_hidden, n_layers, drop_prob).to(device)
hidden = None
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)


In [213]:
model.inference("Улитка заходит в бар ", device=device)


'Улитка заходит в бар ветбебебебеновет<<<больбольннениеНаттНатротНаттНаfНаНароНаттНаfНаНароНаттНаfНаНароНаттНаfНаНароНаттНаfНаНароНаттНаfНаНароНаттНаfНаНароНаттНаfНаНароНаттНаfНаНароНаттНаfНа'

In [214]:
def plot_losses(losses):
    clear_output()
    plt.plot(range(1, len(losses) + 1), losses)
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.show()

In [215]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dataset = JokesDataset(tokenizer, cut_text, 256)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

In [216]:
losses = []
num_epochs = 3

for epoch in range(1, num_epochs + 1):
    epoch_loss = 0
    for i, batch in enumerate(dataloader):
        loss = training_step(model, batch, tokenizer.vocab_size, criterion, optimizer, device)
        epoch_loss += loss
        
        if i % 100 == 0:
            print(f'Done {i/len(dataloader) * 100:.2f}%, Loss: {loss:.4f}')
    epoch_loss /= len(dataloader)
    losses.append(epoch_loss)
    
    plot_losses(losses)
    torch.save(model.state_dict(), "rnn.pt")

RuntimeError: For batched 3-D input, hx and cx should also be 3-D but got (0-D, 0-D) tensors

In [196]:
'''losses = []
num_epochs = 3

for epoch in range(1, num_epochs + 1):
    epoch_loss = 0
    hidden = model.init_hidden(batch_size, device)
    
    print(len(dataloader))
    counter = 0
    for batch in dataloader:
        hidden = tuple([each.data for each in hidden])
        if hidden[0].dim() == 2:
            hidden = (hidden[0].unsqueeze(0), hidden[1].unsqueeze(0))
        loss = training_step(model, batch, tokenizer.vocab_size, criterion, optimizer, device)
        epoch_loss += loss
        counter += 1
        if counter % 100 == 0:
            print(counter)
        
    avg_loss = epoch_loss / len(dataloader)
    losses.append(avg_loss)
    
    print(f'Epoch {epoch}, Loss: {avg_loss:.4f}')
    plot_losses(losses)
    torch.save(model.state_dict(), "rnn.pt")'''


970


RuntimeError: For batched 3-D input, hx and cx should also be 3-D but got (0-D, 0-D) tensors

In [ ]:
[model.inference("Однажды ", device=device) for _ in range(10)]

## {*} Задача 1.1 2 балла
Напишите свой токенайзер вручную, с использованием только библиотек numpy, torch, sklearn, stats, опционально других пакетов, не предоставляющих готовые инструменты токенизации и т.п., за исключением предобработки текста (лемматизация, стеминг и т.д.) . 

Напишем токенайзер, который будет разбивать текст на слова и сохранять их в словарь, а также добавлять специальные токены для начала и конца предложения.

In [217]:
class WordTokenizer:
    def __init__(self, text, max_len=512):
        self.text = text
        self.max_len = max_len
        self.specials = ['<pad>', '<bos>', '<eos>']
        
        # Создаем словарь уникальных слов
        words = set(text.split())
        self.int2word = {i: word for i, word in enumerate(words)}
        self.word2int = {word: i for i, word in self.int2word.items()}
        
        # Добавляем специальные токены
        self._add_special("<pad>")
        self._add_special("<bos>")
        self._add_special("<eos>")

    def _add_special(self, symbol):
        idx = len(self.int2word)
        self.int2word[idx] = symbol
        self.word2int[symbol] = idx

    @property
    def vocab_size(self):
        return len(self.int2word)

    def encode(self, sentence):
        words = ['<bos>'] + sentence.split() + ['<eos>']
        return [self.word2int.get(word, self.word2int['<pad>']) for word in words]

    def decode(self, indices):
        words = [self.int2word.get(index, '<pad>') for index in indices]
        return ' '.join(words).replace('<bos> ', '').replace(' <eos>', '')

In [219]:
 with open(r"../../additional_materials/anek_djvu.txt", "r", encoding="utf-8") as f:
       text = f.read()

# Пример использования
tokenizer = WordTokenizer(text)

encoded = tokenizer.encode("Это пример текста для токенизации. Это еще один пример.")
print("Encoded:", encoded)

decoded = tokenizer.decode(encoded)
print("Decoded:", decoded)

Encoded: [322452, 268037, 43138, 52361, 267605, 322451, 268037, 164829, 321060, 52151, 322453]
Decoded: Это пример текста для <pad> Это еще один пример.


## Задача 2. 4 балла
Реализуйте с помощью только torch/numpy слой RNN, обучите его на данных из классной работы и, опционально, своих данных. Покажите, что модель обучается. 

In [221]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

class SimpleRNNLayer:
    def __init__(self, input_size, hidden_size, output_size):
        self.hidden_size = hidden_size
        
        # Инициализация весов
        self.Wx = torch.tensor(np.random.randn(input_size, hidden_size) * 0.01, dtype=torch.float32, requires_grad=True)
        self.Wh = torch.tensor(np.random.randn(hidden_size, hidden_size) * 0.01, dtype=torch.float32, requires_grad=True)
        self.Wy = torch.tensor(np.random.randn(hidden_size, output_size) * 0.01, dtype=torch.float32, requires_grad=True)
        
        self.bh = torch.zeros((1, hidden_size), dtype=torch.float32, requires_grad=True)
        self.by = torch.zeros((1, output_size), dtype=torch.float32, requires_grad=True)
        
    def forward(self, x, h_prev):
        # Прямой проход
        h_next = torch.tanh(x @ self.Wx + h_prev @ self.Wh + self.bh)
        y = h_next @ self.Wy + self.by
        return y, h_next

    def init_hidden(self, batch_size):
        # Инициализация скрытого состояния
        return torch.zeros((batch_size, self.hidden_size), dtype=torch.float32)

# Пример датасета
class SequenceDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Пример данных
num_samples = 1000
seq_length = 10
input_size = 5
hidden_size = 20
output_size = 3

data = torch.randn(num_samples, seq_length, input_size)
labels = torch.randint(0, output_size, (num_samples,))

dataset = SequenceDataset(data, labels)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Инициализация модели
rnn_layer = SimpleRNNLayer(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD([rnn_layer.Wx, rnn_layer.Wh, rnn_layer.Wy, rnn_layer.bh, rnn_layer.by], lr=0.01)

# Цикл обучения
num_epochs = 10
for epoch in range(num_epochs):
    total_loss = 0
    for batch_data, batch_labels in dataloader:
        batch_size = batch_data.size(0)
        h_prev = rnn_layer.init_hidden(batch_size)
        
        optimizer.zero_grad()
        
        # Прямой проход
        for t in range(seq_length):
            y_pred, h_prev = rnn_layer.forward(batch_data[:, t, :], h_prev)
        
        # Вычисление потерь
        loss = criterion(y_pred, batch_labels)
        total_loss += loss.item()
        
        # Обратный проход и оптимизация
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader):.4f}')

Epoch 1/10, Loss: 1.0986
Epoch 2/10, Loss: 1.0985
Epoch 3/10, Loss: 1.0983
Epoch 4/10, Loss: 1.0982
Epoch 5/10, Loss: 1.0984
Epoch 6/10, Loss: 1.0982
Epoch 7/10, Loss: 1.0980
Epoch 8/10, Loss: 1.0976
Epoch 9/10, Loss: 1.0978
Epoch 10/10, Loss: 1.0975


## Задача 3. 4/5/6/7 баллов
**TBD**: 
Попробуйте обучить рекуррентную сеть задаче классификации. Вы можете воспользоваться сторонними библиотеками для вашей работы, 
но модель и основной код должны быть написаны на pytorch. 

In [178]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np

# Пример датасета
class SequenceDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Пример данных
num_samples = 1000
seq_length = 10
input_size = 5
num_classes = 3

#data = torch.randn(num_samples, seq_length, input_size)
#labels = torch.randint(0, num_classes, (num_samples,))

#dataset = SequenceDataset(data, labels)
#dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
dataset = JokesDataset(tokenizer, cut_text, 256)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

# Определение модели
class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(RNNClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])  # Используем последнее скрытое состояние
        return out

# Инициализация модели, функции потерь и оптимизатора
hidden_size = 20
model = RNNClassifier(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Цикл обучения
num_epochs = 10
for epoch in range(num_epochs):
    for batch_data, batch_labels in dataloader:
        optimizer.zero_grad()
        outputs = model(batch_data)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')


TypeError: full() received an invalid combination of arguments - got (int, int, dtype=torch.dtype), but expected one of:
 * (tuple of ints size, Number fill_value, *, tuple of names names, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (tuple of ints size, Number fill_value, *, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)


In [166]:
# Пример предсказания
with torch.no_grad():
    sample_data = torch.randn(1, seq_length, input_size)
    prediction = model(sample_data)
    predicted_class = torch.argmax(prediction, dim=1)
    print(f'Predicted class: {predicted_class.item()}')

Predicted class: 1


##  {*} Задача 4. [5/6/7/8] баллов
[ссылка](https://www.kaggle.com/t/b2ef08dc3ddf44f981e2ad186c6c508d)

Попробуйте обучить сверточную нейронную сеть задаче детекции людей на изображениях разного стиля. Вы можете воспользоваться сторонними библиотеками для вашей работы. Однако, за неисопользование полностью готовых скриптов обучения (как в классной работе) вы получите дополнительные2 балла

In [79]:
import os
from PIL import Image
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

# Определяем трансформации для изображений
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Изменяем размер изображений
    transforms.ToTensor(),  # Преобразуем в тензор
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Нормализуем изображения
])

# Создаем собственный датасет
class ImageDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.transform = transform
        self.image_files = [f for f in os.listdir(image_folder) if f.endswith(('jpg', 'jpeg', 'png'))]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_folder, self.image_files[idx])
        image = Image.open(img_name).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image

# Путь к папке с изображениями
image_folder = '../additional_materials/images_dataset/'

# Создаем экземпляр датасета
dataset = ImageDataset(image_folder, transform=transform)

# Создаем DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

# Пример использования DataLoader
for images in dataloader:
    print(images.shape)  # (batch_size, 3, 128, 128)
    break

torch.Size([32, 3, 128, 128])


In [ ]:
# TRASH

import numpy as np
import torch
from collections import Counter

class SimpleTokenizer:
    def __init__(self, text, max_vocab_size=322500):
        self.text = text
        self.max_vocab_size = max_vocab_size
        self.special_tokens = ['<pad>', '<bos>', '<eos>', '<unk>']
        
        # Создаем словарь частотности слов
        self.word_counts = Counter(self.text.split())
        
        # Ограничиваем размер словаря
        self.vocab = self.special_tokens + [word for word, _ in self.word_counts.most_common(max_vocab_size - len(self.special_tokens))]
        
        # Создаем отображения слов в индексы и обратно
        self.word2idx = {word: idx for idx, word in enumerate(self.vocab)}
        self.idx2word = {idx: word for word, idx in self.word2idx.items()}

        # Добавляем специальные токены
        self._add_special("<pad>")
        self._add_special("<bos>")
        self._add_special("<eos>")
        self._add_special("<unk>") # неизвестеые слова (не были включены в словарь токенайзера)

    def _add_special(self, symbol):
        idx = len(self.idx2word)
        self.idx2word[idx] = symbol
        self.word2idx[symbol] = idx

    #@property
    def encode(self, sentence):
        # Преобразуем предложение в список индексов
        words = sentence.split()
        indices = [self.word2idx.get(word, self.word2idx['<unk>']) for word in words]
        return [self.word2idx['<bos>']] + indices + [self.word2idx['<eos>']]

    def decode(self, indices):
        # Преобразуем список индексов обратно в предложение
        words = [self.idx2word.get(idx, '<unk>') for idx in indices]
        return ' '.join(words).replace('<bos> ', '').replace(' <eos>', '')

    def pad_sequence(self, sequence, max_length):
        # Дополняем последовательность до максимальной длины
        if len(sequence) < max_length:
            sequence += [self.word2idx['<pad>']] * (max_length - len(sequence))
        return sequence[:max_length]


# Пример использования
tokenizer = SimpleTokenizer(text)

encoded = tokenizer.encode("Это пример текста для токенизации. Это еще один пример.")
print("Encoded:", encoded)

decoded = tokenizer.decode(encoded)
print("Decoded:", decoded)

# Пример дополнения последовательности
padded_sequence = tokenizer.pad_sequence(encoded, 20)
print("Padded:", padded_sequence)